# CIFAR-10 using Residual Neural Networks

## Importing Libraries and Packages

In [1]:
import numpy as np 
import pandas as pd 
from tensorflow import keras
from keras import Model, Input
from keras.datasets import cifar10
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Add, Activation
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Loading and Preparing Data

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Before moving on let's normalize the imgae data so that it is between 0 and 1.

In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

As the labels are intergers let's convert them into array for simpler processing.

In [5]:
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

## Defining functions for Convolution blocks and Building Model 

In [6]:
def convolution_block(X, filters, f = 3):
    X_shortcut = X
    F1, F2 = filters 
    
    # For the convolutional part
    X = Conv2D(filters = F1  , kernel_size=(f,f), activation = "relu", padding = "Same")(X)
    X = Conv2D(filters = F2, kernel_size=(f,f), activation = "relu", padding = "Same")(X)

    # For the shortcut path
    X_shortcut = Conv2D(filters = F2, kernel_size=(1,1), activation = "relu", padding = "Same")(X_shortcut)

    #
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    return X

In [7]:
def res_net(input_shape = (32,32,3), classes = 10) :
    X_input = Input(input_shape)
    
    #Convolution Block 1
    X = convolution_block(X_input, filters = [3,6] , f = 5)
    X = MaxPooling2D(pool_size=(2,2), strides =2)(X)
    
    #Convolution Block 2
    X = convolution_block(X, filters = [6,16] , f = 5)
    X = MaxPooling2D(pool_size=(2,2), strides =2)(X)
    X = BatchNormalization(axis = 3)(X)
    
    #Convolution Block 3
    X = convolution_block(X, filters = [16,32] , f = 3)
    X = MaxPooling2D(pool_size=(2,2), strides =2)(X)

    #Convolution Block 4
    X = Conv2D(64, kernel_size = (3,3), activation = 'relu', padding = "Valid")(X)
    X = BatchNormalization(axis = 3)(X)
    
    X = Flatten()(X)
    X = Dense(32, activation = 'relu')(X)
    X = BatchNormalization(axis = 1)(X)
    X_output = Dense(10, activation = 'softmax')(X)
    
    model = Model(X_input, X_output)
    
    return model

Let's name the model and see it's structure and other important architectural details.

In [8]:
model = res_net(input_shape = (32,32,3), classes = 10)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 3)    228         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 32, 32, 6)    24          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 6)    456         conv2d_1[0][0]                   
____________________________________________________________________________________________

## Training and testing the model
This code below defines the important hyperparameters for training of model.<br>
We're using learning rate decay for smoother and accurate training of model.

In [9]:
batch_size = 256
epochs = 20
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

Compiling and fitting the model.

In [10]:
model.compile(loss="categorical_crossentropy", optimizer=optimizer , metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.0)

Epoch 1/20
50000/50000 [==============================] - 46s 923us/step - loss: 1.6222 - accuracy: 0.4088
Epoch 2/20
50000/50000 [==============================] - 44s 886us/step - loss: 1.2936 - accuracy: 0.5326
Epoch 3/20
50000/50000 [==============================] - 44s 886us/step - loss: 1.1634 - accuracy: 0.5830
Epoch 4/20
50000/50000 [==============================] - 45s 895us/step - loss: 1.0596 - accuracy: 0.6245
Epoch 5/20
50000/50000 [==============================] - 46s 923us/step - loss: 0.9847 - accuracy: 0.6523
Epoch 6/20
50000/50000 [==============================] - 47s 934us/step - loss: 0.9288 - accuracy: 0.6714
Epoch 7/20
50000/50000 [==============================] - 44s 876us/step - loss: 0.8768 - accuracy: 0.6907
Epoch 8/20
50000/50000 [==============================] - 44s 876us/step - loss: 0.8339 - accuracy: 0.7066
Epoch 9/20
50000/50000 [==============================] - 43s 868us/step - loss: 0.8020 - accuracy: 0.7176
Epoch 10/20
50000/50000 [============

Checking the accuracy of the model.

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.960006473827362
Test accuracy: 0.6926000118255615
